In [58]:
import pandas as pd
import numpy as np
import pyodbc
import comm as cmm
import time
import codecs
import os
from string import Template

## sep變數：切割字元

def fmtViewColumns(r):
    sep, jsCol, colDscr, dataType, isOrderBy = "\n\t\t", r["jsCol"], r["ColDscr"], r["Type"], r["orderBy"]
    dataType = "right" if (dataType in ["int","float","decical"]) else "left"
    field = "(row) => dayjs(row.updateDt).format('YYYY-MM-DD HH:mm:ss')" if(jsCol == "updateDt") else "'" + jsCol + "'"
    tmp = Template("{$sep name: '$jsCol',$sep label: '$colDscr',$sep align: '$dataType',$sep field: $field,$sep sortable: true,\n},\n")
    return tmp.substitute(sep = sep, jsCol = jsCol, colDscr= colDscr, dataType = dataType, field = field) if(isOrderBy =="V") else ""
    
def fmtViewFilterItem(r):
    sep, col, colDscr, filterItem = "\n\t\t", r["Column"], r["ColDscr"], r["filterItem"]
    tmp = Template("{$sep label: '$colDscr',$sep value: '$col',$sep placeholder: '請輸入$colDscr',\n},\n")
    return tmp.substitute(sep = sep ,col = col, colDscr= colDscr) if(filterItem =="V") else ""

def fmtEditFormIniValue(r):
    split, col, type, val = "\t\t\t", r["Column"], r["Type"], ""
    if type in ["int","float","decimal"]:
        val = "0"
    elif type in ["varchar","nvarchar","text"]:
        val = "''"
    elif type in ["datetime2","datetime"]:
        val = "''"
    elif type in ["bit"]:
        val = "true"
    else:
        val="''"

    tmp = Template("$split$col: $val,\n")
    return tmp.substitute(split = split, col = col, val = val) if col not in ["creator","createDT","updator","updateDt"] else ""

def fmtEditForm_q_Input(r):
    split, qinput, col, colDscr, maxLen = "\t\t\t", r["q-input"], r["Column"], r["ColDscr"], r["maxLen"]
    tmp = f"""
                <q-input
                    class="col"
                    type="text"
                    v-model.trim="formData.{col}"
                    label="{colDscr}"
                    lazy-rules
                    :rules="[useRequiredInput, (val) => useMaxLength(val, {maxLen})]"
                />"""
    return "" if (qinput == "") else tmp.format(col = col, colDscr = colDscr, maxLen = maxLen)


def copyJsFile(tbName,fName):
    tmp_editForm = codecs.open(f"template/JS/store/admin/{fName}.js",mode="r",encoding="utf-8", buffering=-1).read()
    t = MyTemplate(tmp_editForm).substitute(pt_TableName = tbName)
    fileName = f"JS/src/store/admin/{tbName}/{fName}.js"
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    f = codecs.open(fileName, mode="w", encoding="utf-8", errors="strict")
    f.write(t)
    f.close()

def genJSCode(tb, dfm, dfc):
    pt_TableName, pt_tableName, pt_table_name = dfm["TableName"].values[0], dfm["tableName"].values[0], dfm["table_name"].values[0]

    target = dfc[dfc["orderBy"] == "V"]
    target = target[target["Column"]!="updateDt"]

    pt_indexVue_Columns = "".join(target["indexVue_Columns"].tolist())[:-1]
    pt_filterItem = "".join(target["filterItem2"].tolist())[:-1]
    pt_editForm_formData = "".join(dfc["defaultInitValue"].tolist())[1:-1]

    df_q = dfc[dfc["q-input"] != ""]
    iMax = int(df_q["q-input"].max())
    pt_EditForm_q_input =""
    for q in range(iMax):
        k, ss = q+1, ""
        for e in df_q[df_q["q-input"]==k]["qInputStr"].tolist():
            ss += e

        pt_EditForm_q_input += f"""
            <div class="row q-col-gutter-md">
                {ss}
            </div>""".format(ss = ss)
    
def fmtJSColumn(r):
    col = r["Column"]
    return col[0:1].lower() + col[1:]

def fmtDotNetType(r):
    t, typ = r["Type"], ""
    if t in ["int","float","decimal"]:
        typ = "int"
    elif t in ["varchar","nvarchar","text"]:
        typ = "string"
    elif t in ["datetime","datetime2"]:
        typ = "DateTime"
    elif t == "bit":
        typ = "bool"
    return typ


def fmtApiModels(r):
    col, colDscr, typ, dotnetType, isNull = r["Column"], r["ColDscr"], r["Type"], r["dotnetType"], r["isNull"]
    isNull_a = "?" if (isNull == "V") else ""
    isNull_b = " = null!;" if (isNull == "" and typ in ["varchar","nvarchar"]) else ""

    tmp = Template("""
        /// <summary>
        /// $colDscr
        /// </summary>
        public $dotnetType$isNull_a $col { get; set; } $isNull_b""")

    return tmp.substitute(col = col, colDscr = colDscr, dotnetType = dotnetType, isNull_a = isNull_a, isNull_b = isNull_b)

def fmtApiModelsJs(r):
    jsCol, colDscr, typ, dotnetType, isNull = r["jsCol"], r["ColDscr"], r["Type"], r["dotnetType"], r["isNull"]
    isNull_a = "?" if (isNull == "V") else ""
    isNull_b = " = null!;" if (isNull == "" and typ in ["varchar","nvarchar"]) else ""

    tmp = Template("""
        /// <summary>
        /// $colDscr
        /// </summary>
        public $dotnetType$isNull_a $jsCol { get; set; }""")

    return tmp.substitute(jsCol = jsCol, colDscr = colDscr, dotnetType = dotnetType, isNull_a = isNull_a, isNull_b = isNull_b)

def fmtDotNetDscr(r):
    jsCol, colDscr, dotnetType = r["jsCol"], r["ColDscr"], r["dotnetType"]
    jsCol = '{:<16}'.format(jsCol)
    dotnetType = '{:<10}'.format(dotnetType)

    tmp = Template("\t\t/// $jsCol - $dotnetType - $colDscr\n")
    return tmp.substitute(jsCol = jsCol, colDscr = colDscr, dotnetType = dotnetType)

def fmtPkParasInput(r):
    jsCol, dotnetType, isNull = r["jsCol"], r["dotnetType"] , r["isNull"]
    isNull_a = "?" if (isNull == "V") else ""
    return Template("$dotnetType$isNull_a $jsCol").substitute(jsCol = jsCol, dotnetType = dotnetType, isNull_a =isNull_a)

def fmtData2Json(r):
    sep, col, jsCol = "\t\t\t\t", r["Column"], r["jsCol"] 
    return Template("$sep$jsCol = data.$col,\n").substitute(sep = sep, col = col, jsCol = jsCol)

def fmtItem2Json(r):
    sep, col, jsCol = "\t\t\t\t", r["Column"], r["jsCol"]
    return Template("$sep$jsCol = item.$col,\n").substitute(sep = sep, col = col, jsCol = jsCol)

def fmtJson2Data(r):
    sep, col, jsCol = "\t\t\t\t\t", r["Column"], r["jsCol"]
    return Template("$sep$col = $jsCol,\n").substitute(sep = sep, col = col, jsCol = jsCol)

def fmtJson2Json(r):
    jsCol = r["jsCol"]
    return Template("$jsCol = $jsCol").substitute(jsCol = jsCol)

def fmtRequest2Json(r):
    jsCol = r["jsCol"]
    return Template("request.$jsCol").substitute(jsCol = jsCol)

def fmtInsertRequestItem(r):
    jsCol, dotnetType, colDscr, isNull = r["jsCol"], r["dotnetType"] , r["ColDscr"], r["isNull"]
    isNull_a = "?" if (isNull == "V") else ""

    tmp ="""/// <summary>
    /// $colDscr
    /// </summary>
    public $dotnetType$isNull_a $jsCol { get; set; }"""
    return Template(tmp).substitute(jsCol = jsCol, colDscr = colDscr, dotnetType = dotnetType, isNull_a = isNull_a)

def fmtInsertValidator(r):
    jsCol, colDscr, maxLen = r["jsCol"] ,r["ColDscr"], r["maxLen"]
    tmp = """\t\t\tRuleFor(x => x.$jsCol).NotEmpty().WithMessage("$colDscr為必填");\n\t\t\tRuleFor(x => x.$jsCol).MaximumLength($maxLen).WithMessage("$colDscr長度超過系統限制");\n"""
    return Template(tmp).substitute(jsCol = jsCol, colDscr = colDscr, maxLen = maxLen)

def fmtValidateMustHave(r):
    jsCol, colDscr, maxLen = r["jsCol"] ,r["ColDscr"], r["maxLen"]
    tmp = """\t\t\tRuleFor(x => x.$jsCol).NotEmpty().WithMessage("$colDscr不能為空!!");\n"""
    return Template(tmp).substitute(jsCol = jsCol, colDscr = colDscr, maxLen = maxLen)

def fmtQueryRequest(r):
    jsCol, dotnetType, colDscr, isNull, reqDefaultVal = r["jsCol"], r["dotnetType"] , r["ColDscr"], r["isNull"], r["reqDefaultVal"]
    isNull_a = "?" if (isNull == "V") else ""
    reqDefaultVal = reqDefaultVal if (dotnetType in ["string","DateTime"]) else f" = \"{reqDefaultVal}\""

    tmp ="""\t\t/// <summary>
        /// $colDscr
        /// </summary>
        public $dotnetType$isNull_a $jsCol { get; set; } $reqDefaultVal\n"""
    return Template(tmp).substitute(jsCol = jsCol, colDscr = colDscr, dotnetType = dotnetType, isNull_a = isNull_a, reqDefaultVal = reqDefaultVal)

def fmtQueryResponse(r):
    jsCol, dotnetType, colDscr, isNull = r["jsCol"], r["dotnetType"] , r["ColDscr"], r["isNull"]
    isNull_a = "?" if (isNull == "V") else ""

    tmp ="""\t\t/// <summary>
        /// $colDscr
        /// </summary>
        public $dotnetType$isNull_a $jsCol { get; set; }\n"""
    return Template(tmp).substitute(jsCol = jsCol, colDscr = colDscr, dotnetType = dotnetType, isNull_a = isNull_a)


    ## -----套表區
    tmpJSFile = codecs.open(f"template/JS/views/admin/index.vue",mode="r",encoding="utf-8", buffering=-1).read()
    t1 = Template(tmpJSFile).substitute(pt_TableName = pt_TableName, pt_tableName = pt_tableName, pt_table_name = pt_table_name, pt_indexVue_Columns = pt_indexVue_Columns, pt_filterItem = pt_filterItem)
    fileName = f"JS/src/views/admin/{pt_table_name}/index.vue"
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    f = codecs.open(fileName, mode="w", encoding="utf-8", errors="strict")
    f.write(t1)
    f.close()

    tmp_editForm = codecs.open(f"template/JS/views/admin/edit-form.vue",mode="r",encoding="utf-8", buffering=-1).read()
    t2 = MyTemplate(tmp_editForm).substitute(pt_editForm_formData = pt_editForm_formData, pt_EditForm_q_input = pt_EditForm_q_input)
    fileName = f"JS/src/views/admin/{pt_table_name}/edit-form.vue"
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    f = codecs.open(fileName, mode="w", encoding="utf-8", errors="strict")
    f.write(t2)
    f.close()

    tmp_actionsForm = codecs.open(f"template/JS/store/admin/actions.js",mode="r",encoding="utf-8", buffering=-1).read()
    t3 = MyTemplate(tmp_actionsForm).substitute(pt_TableName = pt_TableName)
    fileName = f"JS/src/store/admin/{pt_table_name}/actions.js"
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    f = codecs.open(fileName, mode="w", encoding="utf-8", errors="strict")
    f.write(t3)
    f.close()

    copyJsFile(pt_table_name, "getters")
    copyJsFile(pt_table_name, "index")
    copyJsFile(pt_table_name, "mutations")
    copyJsFile(pt_table_name, "state")


In [63]:
from string import Template

pd.options.mode.chained_assignment = None  # default='warn'
class MyTemplate(Template):
    delimiter = "//$"           #自己定義的替換字元符號

# class AtTemplate(Template):
#     delimiter = "@"            #自己定義的替換字元符號

df_sp = pd.read_excel("tableSourceDef.xlsx", sheet_name="SP_config",converters={'maxLen':str }).replace(np.nan, '')
tbs = df_sp["TableName"].unique()

df = pd.read_excel("tableSourceDef.xlsx", sheet_name="admin_columns",converters={'maxLen':str }).replace(np.nan, '')
df["jsCol"] = df.apply(fmtJSColumn, axis =1)                  # PcPhone 轉為 pcPhone 
df["indexVue_Columns"] = df.apply(fmtViewColumns, axis = 1)   # {name:'extCode',label:'分機',align:left,field:'extCode',sor
df["filterItem2"] = df.apply(fmtViewFilterItem, axis = 1)     #
df["defaultInitValue"] = df.apply(fmtEditFormIniValue, axis = 1)
df["qInputStr"] = df.apply(fmtEditForm_q_Input, axis = 1)
df["dotnetType"] = df.apply(fmtDotNetType, axis = 1)          #Sql資料型別轉換為 .net資料型別
df["api_Models"] = df.apply(fmtApiModels, axis = 1)
df["api_DotnetDscr"] = df.apply(fmtDotNetDscr, axis = 1)       #.net內的欄位說明
df["api_PkParasInput"] = df.apply(fmtPkParasInput, axis =1)
df["api_Data2Json"] = df.apply(fmtData2Json, axis = 1)         #轉換為Json
df["api_Item2Json"] = df.apply(fmtItem2Json, axis = 1)         #轉換為Json 
df["api_Json2Data"] = df.apply(fmtJson2Data, axis = 1)         #轉換為Json 
df["api_Json2Json"] = df.apply(fmtJson2Json, axis = 1)  
df["api_Request2Json"] = df.apply(fmtRequest2Json, axis = 1)  
df["api_ApiModelsJs"] = df.apply(fmtApiModelsJs, axis = 1)     #///<summary>分機</summary>public string? extCode { get;...   
df["api_InsertValidator"] = df.apply(fmtInsertValidator, axis = 1)
df["api_ValidateMustHave"] = df.apply(fmtValidateMustHave, axis = 1)
df["api_QueryRequest"] = df.apply(fmtQueryRequest, axis = 1)
df["api_QueryResponse"] = df.apply(fmtQueryResponse, axis = 1)


def genOneWebApiCode(source, target, _dictParas):
    _source = "template/WebApi/" + source
    fileName = f"WebApi/" + target
    # print("source:", _source)
    # print("target:", fileName)

    tmp = codecs.open(_source, mode="r",encoding="utf-8", buffering=-1).read()
    t = Template(tmp).substitute(_dictParas)
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    f = codecs.open(fileName, mode="w", encoding="utf-8", errors="strict")
    f.write(t)
    f.close()


def genWebApiCode(tb, dfm, dfc):
    pt_TableName, pt_tableName, pt_table_name = dfm["TableName"].values[0], dfm["tableName"].values[0], dfm["table_name"].values[0]
    TbDscr = dfm["tbDscr"].values[0]

    pt_apiModels = "".join(dfc["api_Models"].tolist())
    pt_apiModels_insertRequest = "".join(dfc[dfc["httpInsert"] != ""]["api_ApiModelsJs"].tolist())
    pt_InputPK = ",".join(dfc[dfc["PK"] == "V"]["api_PkParasInput"].tolist())
    pt_InputIsExist = ",".join(dfc[dfc["isExist"] != ""]["api_PkParasInput"].tolist())

    pt_InputServicePK = ",".join(dfc[dfc["PK"] == "V"]["Column"].tolist())
    pt_ColDscr_ParasPK = "".join(dfc[dfc["PK"] == "V"]["api_DotnetDscr"].tolist())[:-1]
    pt_ColDscr_GetReturnAll = "".join(dfc["api_DotnetDscr"].tolist())[:-1]
    pt_ColDscr_QueryParas = "".join(dfc[dfc["httpInsert"] != ""]["api_DotnetDscr"].tolist())[:-1]
    pt_ColDscr_InsertParas = "".join(dfc[dfc["httpInsert"] != ""]["api_DotnetDscr"].tolist())[:-1]
    pt_colDscr_UpdateParas = "".join(dfc[dfc["httpUpdate"] != ""]["api_DotnetDscr"].tolist())[:-1]
    pt_colDscr_ExistsParas = "".join(dfc[dfc["isExist"] != ""]["api_DotnetDscr"].tolist())[:-2]
    
    pt_data2Json = "".join(dfc["api_Data2Json"].tolist())[:-1]
    pt_item2Json = "".join(dfc["api_Item2Json"].tolist())[:-1]
    pt_json2Data = "".join(dfc[dfc["isExist"] != ""]["api_Json2Data"].tolist())[:-1]
    pt_json2json = ", ".join(dfc[dfc["PK"] == "V"]["api_Json2Json"].tolist())
    pt_request2Json = ", ".join(dfc[dfc["isExist"] != ""]["api_Request2Json"].tolist())

    pt_insertValidator = "".join(dfc[(dfc["Type"].isin(["varchar","nvarchar"])) & (dfc["isNull"] != "V") & (dfc["Column"] != "Creator") ]["api_InsertValidator"].tolist())[:-1]
    pt_ValidateMustHave ="".join(dfc[ dfc["PK"] == "V"]["api_ValidateMustHave"].tolist())[:-1]
    pt_queryRequest = "".join(dfc[dfc["queryWhere"] != ""]["api_QueryRequest"].tolist())[:-1]
    pt_queryResponse = "".join(dfc[dfc["queryWhere"] != ""]["api_QueryResponse"].tolist())[:-1]
    pt_updateRequest = "".join(dfc[dfc["httpUpdate"] != ""]["api_QueryResponse"].tolist())[:-1]
    # print(pt_insertValidator)

    ## 準備要套用變數。
    dictParas = dict(pt_TableName = pt_TableName, TbDscr = TbDscr, controller="Controller", service="Service", queryRequest = "QueryRequest", insertRequest="InsertRequest", insertValidator = "InsertValidator", updateRequest = "UpdateRequest", updateValidator = "UpdateValidator", get = "Get", response ="Response", request ="Request", exists = "Exists", query = "Query", insert = "Insert", update = "Update", delete = "Delete", pt_apiModels = pt_apiModels, pt_ColDscr_ParasPK = pt_ColDscr_ParasPK, pt_InputPK = pt_InputPK, pt_InputServicePK = pt_InputServicePK,  pt_ColDscr_GetReturnAll = pt_ColDscr_GetReturnAll , pt_ColDscr_QueryParas = pt_ColDscr_QueryParas, pt_ColDscr_InsertParas = pt_ColDscr_InsertParas, pt_colDscr_UpdateParas = pt_colDscr_UpdateParas, pt_colDscr_ExistsParas = pt_colDscr_ExistsParas, pt_InputIsExist = pt_InputIsExist , pt_data2Json = pt_data2Json, pt_item2Json = pt_item2Json, pt_json2Data = pt_json2Data, pt_json2json = pt_json2json, pt_request2Json = pt_request2Json, pt_apiModels_insertRequest = pt_apiModels_insertRequest, pt_insertValidator = pt_insertValidator, pt_ValidateMustHave = pt_ValidateMustHave, pt_queryRequest = pt_queryRequest, pt_queryResponse = pt_queryResponse, pt_updateRequest = pt_updateRequest)

    ## 開始套表...
    genOneWebApiCode("Models/TbPcPhone.cs",                             f"Models/Tb{pt_TableName}.cs", dictParas)
    genOneWebApiCode("Controllers/PcPhoneController.cs",                f"Controllers/{pt_TableName}Controller.cs", dictParas)
    genOneWebApiCode("Application/IPcPhoneService.cs",                  f"Application/{pt_TableName}/I{pt_TableName}Service.cs", dictParas)
    genOneWebApiCode("Application/PcPhoneService.cs",                   f"Application/{pt_TableName}/{pt_TableName}Service.cs", dictParas)
    genOneWebApiCode("Application/Contract/PcPhoneInsertRequet.cs",     f"Application/{pt_TableName}/Contract/{pt_TableName}InsertRequest.cs", dictParas)
    genOneWebApiCode("Application/Contract/PcPhoneInsertValidator.cs",  f"Application/{pt_TableName}/Contract/{pt_TableName}InsertValidator.cs", dictParas) 
    genOneWebApiCode("Application/Contract/PcPhoneQueryRequest.cs",     f"Application/{pt_TableName}/Contract/{pt_TableName}QueryRequest.cs", dictParas) 
    genOneWebApiCode("Application/Contract/PcPhoneResponse.cs",         f"Application/{pt_TableName}/Contract/{pt_TableName}Response.cs", dictParas) 
    genOneWebApiCode("Application/Contract/PcPhoneUpdateRequest.cs",    f"Application/{pt_TableName}/Contract/{pt_TableName}UpdateRequest.cs", dictParas) 
    genOneWebApiCode("Application/Contract/PcPhoneUpdateValidator.cs",  f"Application/{pt_TableName}/Contract/{pt_TableName}UpdateValidator.cs", dictParas) 
 

for tb in tbs:

    if tb in ["PcPhone","Code","Role"]:
        dfm = df_sp[df_sp["TableName"] == tb]
        dfc = df[df["TableName"] == tb]
     
        
        genJSCode(tb, dfm, dfc)
        genWebApiCode(tb, dfm, dfc)
    
